In [1]:
import dspy

/Users/Salim/Library/Caches/pypoetry/virtualenvs/dspy-toy-SOZM3orb-py3.11/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
import os

In [6]:
api_key = os.environ.get('OPENAI_API_KEY')

In [7]:
lm = dspy.LM('openai/gpt-4.1-nano', api_key=api_key)
dspy.configure(lm=lm)

In [16]:
mom_test_examples = [
  {
    "business_proposal": "One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.",
    "scary_question": "Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?"
  },
  {
    "business_proposal": "We suspect that teachers from the poorest schools are completely overloaded, and that our tools would save them time so they could better educate their students.",
    "scary_question": "Do you have a budget allocated for tools like this?"
  },
  {
    "business_proposal": "Bars will pay, but only if you can amass a huge audience of consumers. The founders talk to consumers and ask if they would use an app which always pointed them to booming parties with cheap booze.",
    "scary_question": "How would we reach the critical mass of users needed to make this appealing to bars?"
  },
  {
    "business_proposal": "You’ve got $50k to start your business and you want to know whether your idea is any good before committing all your resources.",
    "scary_question": "What’s the most likely reason this business could fail?"
  },
  {
    "business_proposal": "You’re building a solution for a customer segment that includes 'students' or 'sales organizations,' thinking you’re focused.",
    "scary_question": "Am I talking to a consistent customer segment or are these conversations spread across too many types?"
  }
]


In [14]:
synthetic_examples = [
  {
    "business_proposal": "We’re building a marketplace for local artisans to sell their crafts directly to consumers, cutting out the middleman.",
    "scary_question": "Have you ever tried selling your products online, and what stopped you if you haven’t?"
  },
  {
    "business_proposal": "We want to automate nutrition planning for athletes with an AI coach that updates meal plans daily based on performance.",
    "scary_question": "How do you currently plan your meals and how often do you actually adjust them based on training feedback?"
  },
  {
    "business_proposal": "Our product helps landlords track maintenance requests and rent payments in one dashboard.",
    "scary_question": "What are the biggest problems in your rental workflow right now, and do any of them actually cost you money or tenants?"
  },
  {
    "business_proposal": "We’re launching a subscription box of STEM projects for kids aged 8–12 to foster early interest in engineering.",
    "scary_question": "When was the last time you bought something educational for your kids that wasn’t required by school?"
  },
  {
    "business_proposal": "We're building a tool to help wedding planners coordinate vendors, guests, and schedules in one place.",
    "scary_question": "What’s the worst part of planning a wedding, and how do you currently manage that part?"
  },
  {
    "business_proposal": "We want to help freelancers manage their taxes and automatically save for quarterly payments.",
    "scary_question": "Have you ever missed a tax payment or been hit with a fine? What did you do about it?"
  },
  {
    "business_proposal": "We’re developing a social platform for book lovers to find and join local book clubs.",
    "scary_question": "How do you currently find new people to read or discuss books with, and do you feel like that’s a gap in your life?"
  },
  {
    "business_proposal": "We’re building software to help nonprofit boards manage compliance and governance more easily.",
    "scary_question": "Has governance or compliance ever caused real delays or risks for your nonprofit?"
  },
  {
    "business_proposal": "We want to make a budgeting app for couples to track and manage shared expenses transparently.",
    "scary_question": "What’s the last disagreement you had with your partner about money, and how did you resolve it?"
  },
  {
    "business_proposal": "We’re building an AI tool to help content marketers repurpose long-form blogs into social media posts.",
    "scary_question": "What’s stopping you from turning your best content into social posts today?"
  }
]

In [36]:
trainset = [
    dspy.Example(business_proposal=x.get('business_proposal'), hint=x.get('scary_question'), label=x.get('scary_question')).with_inputs("business_proposal", "hint")
    for x in mom_test_examples + synthetic_examples
]

In [37]:
trainset

[Example({'business_proposal': "One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.", 'hint': 'Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?', 'label': 'Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?'}) (input_keys={'business_proposal', 'hint'}),
 Example({'business_proposal': 'We suspect that teachers from the poorest schools are completely overloaded, and that our tools would save them time so they could better educate their students.', 'hint': 'Do you have a budget allocated for tools like this?', 'label': 'Do you have a budget allocated for tools like this?'}) (input_keys={'business_proposal', 'hint'}),
 Example({'business_pro

In [51]:
import dspy
from dspy.evaluate import SemanticF1
from dspy import settings

settings.use_structured_output = False   # stop adding response_format
settings.rationale_mode        = "none"  # stop adding rationale_type

dspy.configure(lm=dspy.LM('openai/gpt-3.5-turbo-0125'))

sig = dspy.ChainOfThought("business_proposal -> scary_question: str") 
ask_scary = dspy.Predict(sig)
metric = SemanticF1(threshold=0.75)
optim = dspy.BootstrapFinetune(metric=metric, num_threads=8)
scary_agent = optim.compile(ask_scary, trainset=trainset)

2025/05/22 22:06:39 INFO dspy.teleprompt.bootstrap_finetune: Preparing the student and teacher programs...
2025/05/22 22:06:39 INFO dspy.teleprompt.bootstrap_finetune: Bootstrapping data...
2025/05/22 22:06:39 ERROR dspy.utils.parallelizer: Error for Example({'business_proposal': "One of my companies had some legal ambiguities around content ownership. It was okay in theory, but lacked strong precedents. I was always a bit nervous we would get 'found out.' Their execs were excited, their creatives were thrilled. We had even brought the stubborn tech team around to our side.", 'hint': 'Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?', 'label': 'Can I speak with your legal team to understand if they’re comfortable with the content ownership terms?'}) (input_keys={'business_proposal', 'hint'}): 'ChainOfThought' object has no attribute 'input_fields'. Set `provide_traceback=True` for traceback.
2025/05/22 22:06:39 ERROR dspy.utils.par

  0%|                                                                                                                                         | 0/15 [00:00<?, ?it/s]

2025/05/22 22:06:39 ERROR dspy.utils.parallelizer: Error for Example({'business_proposal': "You’re building a solution for a customer segment that includes 'students' or 'sales organizations,' thinking you’re focused.", 'hint': 'Am I talking to a consistent customer segment or are these conversations spread across too many types?', 'label': 'Am I talking to a consistent customer segment or are these conversations spread across too many types?'}) (input_keys={'business_proposal', 'hint'}): 'ChainOfThought' object has no attribute 'input_fields'. Set `provide_traceback=True` for traceback.


Average Metric: 0.00 / 0 (0%):   7%|██████▍                                                                                          | 1/15 [00:00<00:00, 666.71it/s]

2025/05/22 22:06:39 ERROR dspy.utils.parallelizer: Error for Example({'business_proposal': 'We’re building a marketplace for local artisans to sell their crafts directly to consumers, cutting out the middleman.', 'hint': 'Have you ever tried selling your products online, and what stopped you if you haven’t?', 'label': 'Have you ever tried selling your products online, and what stopped you if you haven’t?'}) (input_keys={'business_proposal', 'hint'}): 'ChainOfThought' object has no attribute 'input_fields'. Set `provide_traceback=True` for traceback.


...

Average Metric: 0.00 / 0 (0%):  87%|██████████████████████████████████████████████████████████████████████████████████▎            | 13/15 [00:00<00:00, 1559.71it/s]

2025/05/22 22:06:39 ERROR dspy.utils.parallelizer: Error for Example({'business_proposal': 'We’re building an AI tool to help content marketers repurpose long-form blogs into social media posts.', 'hint': 'What’s stopping you from turning your best content into social posts today?', 'label': 'What’s stopping you from turning your best content into social posts today?'}) (input_keys={'business_proposal', 'hint'}): 'ChainOfThought' object has no attribute 'input_fields'. Set `provide_traceback=True` for traceback.


Average Metric: 0.00 / 0 (0%): 100%|███████████████████████████████████████████████████████████████████████████████████████████████| 15/15 [00:00<00:00, 1639.89it/s]

2025/05/22 22:06:39 INFO dspy.evaluate.evaluate: Average Metric: 0 / 15 (0.0%)
2025/05/22 22:06:39 WARNING dspy.teleprompt.bootstrap_finetune: Failed to unpack prediction and trace. This is likely due to the LLM response not following dspy formatting.


ValueError: not enough values to unpack (expected 2, got 0)

In [ ]:
proposal = """
We’re building an app that lets bars live‑stream trivia so patrons
in different locations can compete in real time.
"""
print(scary_agent(business_proposal=proposal).scary_question)